# Preparação dos dados

In [42]:
# Importação de pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# configurações de visualização
pd.set_option('display.max_columns', None)

## Dados de Apólices

In [ ]:
# Importação dos dados de apólices
df_apolice = pd.read_excel('Apolices_Emitidas_.xlsx')

# Ajustar nome de colunas
df_apolice_colunas_nomes ={
	"nrProposta":						"nr_proposta",
	"nrApolice":						"nr_apolice",
	"nrEndosso":						"nr_endosso",
	"nrEndossoAssociado":				"nr_endosso_associado",
	"dsEndosso":						"ds_endosso",
	"statusApolice":					"status_apolice",
	"idGrupoRamoSeguro":				"id_grp_ramo",
	"RamoSeguro":						"nm_ramo",
	"nmGrupoRamoSeguro":				"nm_grp_ramo",
	"dtEmissaoMovimento":				"dt_emissao_mvto",
	"dtIniVigencia":					"dt_inicio_vigencia",
	"dtFimVigencia":					"dt_fim_vigencia",
	"vlPremioComercialSeguroMoeda":		"vl_premio_comercial",
	"vlCustoAquisicaoOperacionalMoeda":	"vl_custo_aquisicao",
	"vlIOF":							"vl_iof",
	"vlAdicionalFracionamentoMoeda":	"vl_adicional_fracionamento",
	"vlIS":								"vl_is",
	"nrTotalFracionamento":				"nr_total_fracionamento",
	"nrCpfCnpjCorretorLider":			"cpf_cnpj",
	"nmRazaoSocialCorretorLider":		"razao_social",
	"dtNnascimento":					"dt_nascimento",
	"nmSexo":							"sexo",
	"tipoPessoa":						"tipo_pessoa",
	"cdProdutoApolice":					"cd_produto",
	"nmProduto":						"nm_produto",
	"nmCep":							"cep",
	"nmCidade":							"cidade",
	"nmUF":								"uf"
}
df_apolice = df_apolice.rename(columns=df_apolice_colunas_nomes)
memory_usage_before_df_apolice = df_apolice.memory_usage(deep=True)
types_before = df_apolice.dtypes

# Ajustar tipos de colunas
df_apolice_colunas_tipos ={
	"ds_endosso":						"category",
	"status_apolice":					"category",
	"id_grp_ramo":						"category",
	"nm_ramo":							"category",
	"nm_grp_ramo":						"category",
	"dt_emissao_mvto":					"datetime64[ns]",
	"dt_inicio_vigencia":				"datetime64[ns]",
	"dt_fim_vigencia":					"datetime64[ns]",
	"vl_premio_comercial":				"float64",
	"vl_custo_aquisicao":				"float64",
	"vl_iof":							"float64",
	"vl_adicional_fracionamento":		"float64",
	"vl_is":							"float64",
	"nr_total_fracionamento":			"category",
	"cpf_cnpj":							"category",
	"razao_social":						"category",
	"dt_nascimento":					"datetime64[ns]",
	"sexo":								"category",
	"tipo_pessoa":						"category",
	"cd_produto":						"category",
	"nm_produto":						"category",
	"cep":								"string",
	"cidade":							"category",
	"uf":								"category"
	}
df_apolice = df_apolice.astype(df_apolice_colunas_tipos)
memory_usage_after_df_apolice = df_apolice.memory_usage(deep=True)
types_after = df_apolice.dtypes

In [ ]:
# Comparar uso de memória de apólices
memory_usage_df_apolice = pd.DataFrame({
    'types_before': types_before,
	'types_after': types_after,
    'Memory Usage Before': memory_usage_before_df_apolice,
    'Memory Usage After': memory_usage_after_df_apolice,
    'Percentage Difference': (memory_usage_after_df_apolice
                              - memory_usage_before_df_apolice)
                              / memory_usage_before_df_apolice * 100
})
memory_usage_df_apolice.loc['Diferença total de memória'] = [
    '---',
    '---',
    memory_usage_df_apolice['Memory Usage Before'].sum(),
    memory_usage_df_apolice['Memory Usage After'].sum(),
    (memory_usage_df_apolice['Memory Usage After'].sum() 
     - memory_usage_df_apolice['Memory Usage Before'].sum()) 
     / memory_usage_df_apolice['Memory Usage Before'].sum() * 100
]

memory_usage_df_apolice

In [51]:
def analise_pareto(df, colunas, grupo, limite=None):
    for coluna in colunas:
        # Agrupe as variáveis por região
        df_grupo = df.groupby(grupo)[coluna].sum().reset_index()
        
        # Ordene as regiões em ordem decrescente
        df_grupo = df_grupo.sort_values(coluna, ascending=False)
        
        # Se houver um limite, acumule as categorias não exibidas em "Demais"
        if limite is not None:
            df_grupo_demais = df_grupo.iloc[limite:]
            df_grupo_demais = pd.DataFrame({'regiao': ['Demais'], coluna: [df_grupo_demais[coluna].sum()]})
            df_grupo = pd.concat([df_grupo.iloc[:limite], df_grupo_demais])
        
        # Calcule o acumulado de valores
        acumulado = df_grupo[coluna].cumsum()
        
        # Calcule o percentual acumulado
        percentual_acumulado = (acumulado / acumulado.max()) * 100
        
        # Identifique o ponto de Pareto
        ponto_pareto = percentual_acumulado[percentual_acumulado >= 80].index[0]
        
        print(f'Ponto de Pareto para {coluna} por {grupo}: {ponto_pareto}')
        print(f'Valor acumulado até o ponto de Pareto: {acumulado[ponto_pareto]}')
        print(f'Percentual de registros até o ponto de Pareto: {percentual_acumulado[ponto_pareto]}%')
        
        # Crie um gráfico do Plotly
        fig = go.Figure(data=[
            go.Bar(x=df_grupo[grupo], y=df_grupo[coluna], name='Soma dos valores'),
            go.Scatter(x=df_grupo[grupo], y=acumulado, name='Acumulado de Pareto', mode='lines')
        ])
        fig.update_layout(title=f'Análise de Pareto para {coluna} por {grupo}',
                          xaxis_title=grupo,
                          yaxis_title='Valor')
        fig.add_vline(x=df_grupo[grupo][ponto_pareto], line_dash='dash', line_color='red')
        fig.show()

colunas = ['vl_premio_comercial', 'vl_custo_aquisicao']
grupo = 'cidade'
limite = 100
analise_pareto(df_apolice, colunas, grupo, limite)

Ponto de Pareto para vl_premio_comercial por cidade: 710
Valor acumulado até o ponto de Pareto: 248085858.74000004
Percentual de registros até o ponto de Pareto: 80.02984652962866%


C:\Users\frosa\AppData\Local\Temp\ipykernel_47912\2866448968.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Ponto de Pareto para vl_custo_aquisicao por cidade: 309
Valor acumulado até o ponto de Pareto: 72002441.90999997
Percentual de registros até o ponto de Pareto: 80.14937927710048%


C:\Users\frosa\AppData\Local\Temp\ipykernel_47912\2866448968.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Dados de Sinistros

In [ ]:
# Importação dos dados de sinistros
df_sinistro = pd.read_excel('Apolices_Sinistradas_Sicoob_.xlsx')

# Ajustar nome de colunas
df_sinistro_colunas_nomes ={
	"nrApolice":							"nr_apolice",
	"nrEndosso":							"nr_endosso",
	"dsCoberturaSeguro":					"ds_cobertura",
	"CdCoberturaSeguro":					"cd_cobertura",
	"idGrupoRamoSeguro":					"id_grp_ramo",
	"idRamoSeguro":							"id_ramo",
	"nrSinistro":							"nr_sinistro",
	"MovimentoSinistro":					"nm_movto_sinistro",
	"dtEmissaoEventoSinistro":				"dt_emissao_mvto_sinistro",
	"vlEventoSinistroMoeda":				"vl_evento_sinsitro_moeda",
	"vlEventoSinistroBRL":					"vl_evento_sinsitro_brl",
	"dtOcorrenciaSinistro":					"dt_ocorrencia_sinistro",
	"dtAvisoSinistro":						"dt_aviso_sinistro",
	"dtRegistroSinistro":					"dt_registro_sinistro",
	"cdUFOcorrenciaSinistro":				"uf_ocorrencia_sinistro",
	"nrCpfCnpjContraparte":					"cpf_cnpj_contraparte",
	"nmRazaoSocialContraparte":				"razao_social_contraparte",
	"cdProcessoJudicial":					"cd_processo_judicial",
}
df_sinistro = df_sinistro.rename(columns=df_sinistro_colunas_nomes)
memory_usage_before_df_sinistro = df_sinistro.memory_usage(deep=True)
types_before = df_sinistro.dtypes

# Ajustar tipos de colunas
df_sinistro_colunas_tipos ={
	"ds_cobertura":							"category",
	"cd_cobertura":							"category",
	"id_grp_ramo":							"category",
	"id_ramo":								"category",
	"nm_movto_sinistro":					"category",
	"dt_emissao_mvto_sinistro":				"datetime64[ns]",
	"vl_evento_sinsitro_moeda":				"float64",
	"vl_evento_sinsitro_brl":				"float64",
	"dt_ocorrencia_sinistro":				"datetime64[ns]",
	"dt_aviso_sinistro":					"datetime64[ns]",
	"dt_registro_sinistro":					"datetime64[ns]",
	"uf_ocorrencia_sinistro":				"category",
	"cpf_cnpj_contraparte":					"float64",
}
df_sinistro_tipo = df_sinistro.astype(df_sinistro_colunas_tipos)
memory_usage_after_df_sinistro = df_sinistro_tipo.memory_usage(deep=True)
types_after = df_sinistro.dtypes

In [ ]:
# Comparar uso de memória de sinistro
memory_usage_df_sinistro = pd.DataFrame({
    'types_before': types_before,
	'types_after': types_after,
    'Memory Usage Before': memory_usage_before_df_sinistro,
    'Memory Usage After': memory_usage_after_df_sinistro,
    'Percentage Difference': (memory_usage_after_df_sinistro
                              - memory_usage_before_df_sinistro)
                              / memory_usage_before_df_sinistro * 100
})
memory_usage_df_sinistro.loc['Diferença total de memória'] = [
    '---',
    '---',
    memory_usage_df_sinistro['Memory Usage Before'].sum(),
    memory_usage_df_sinistro['Memory Usage After'].sum(),
    (memory_usage_df_sinistro['Memory Usage After'].sum() 
     - memory_usage_df_sinistro['Memory Usage Before'].sum()) 
     / memory_usage_df_sinistro['Memory Usage Before'].sum() * 100
]

memory_usage_df_sinistro